### Part 1

In [1]:
input_file = open('input')
wire1, wire2 = input_file.read().split()
input_file.close()

wire1 = wire1.split(',')
wire2 = wire2.split(',')

In [2]:
import numpy as np

direction = dict()
direction['R'] = np.array((1,0))
direction['L'] = np.array((-1,0))
direction['U'] = np.array((0,1))
direction['D'] = np.array((0,-1))

def parse(instruction):
    dircode = instruction[0]
    length = int(instruction[1:])
    return length*direction[dircode]

def get_corners(wire):
    corners = []
    current_point = np.array((0,0))
    corners.append(np.copy(current_point))
    
    for instruction in wire:
        current_point += parse(instruction)
        corners.append(np.copy(current_point))
    return corners

In [3]:
corners1 = get_corners(wire1)
corners2 = get_corners(wire2)

In [4]:
def manhattan_norm(vector):
    return abs(vector[0]) + abs(vector[1])

def manhattan_distance(vec1, vec2):
    return manhattan_norm(vec1 - vec2)

def between(m, a, b):
    return m >= min(a,b) and m <= max(a,b)

def segseg_intercheck(seg1, seg2, return_ans = False):
    start1, end1 = seg1
    start2, end2 = seg2
    
    const1 = int(start1[0] != end1[0])
    const2 = int(start2[0] != end2[0])
    
    cond1 = between(start1[const1], start2[const1], end2[const1])
    cond2 = between(start2[const2], start1[const2], end1[const2]) 
    
    if return_ans:
        ans = np.array((0,0))
        ans[const1] = start1[const1]
        ans[const2] = start2[const2]
        return ans
    
    return cond1 and cond2

def segpoint_intercheck(seg, point):
    start, end = seg
    
    cond1 = between(point[0], start[0], end[0])
    cond2 = between(point[1], start[1], end[1])
    
    return cond1 and cond2

In [5]:
intersections = []

for i in range(len(corners1)-1):
    for j in range(len(corners2)-1):
        if segseg_intercheck(corners1[i:i+2], corners2[j:j+2]):
            ans = segseg_intercheck(corners1[i:i+2],corners2[j:j+2], return_ans = True)
            if manhattan_norm(ans) > 0:
                intersections.append(ans)

In [12]:
manhattan_distances = [manhattan_norm(vector) for vector in intersections]

inter_index = np.argmin(manhattan_distances)
ans = sorted(manhattan_distances)[0]

print('Closest Intersection is at a', ans, ' manhattan distance')
print('It happens at', tuple(intersections[inter_index]))

Closest Intersection is at a 1064  manhattan distance
It happens at (-37, 1027)


### Part 2

In [7]:
def get_wire_distance(corners, intersection):
    distance = 0
    for i in range(len(corners) -1):
        seg = corners[i:i+2]
        check = segpoint_intercheck(seg, intersection)
        if check:
            distance += manhattan_distance(seg[0], intersection)
            return distance
        else:
            distance += manhattan_distance(seg[0],seg[1])
    print('Intersection never reached')
    return -1

In [8]:
wire_distances1 = [get_wire_distance(corners1, intersection) for intersection in intersections]
wire_distances2 = [get_wire_distance(corners2, intersection) for intersection in intersections]

In [9]:
combined_wire_distances = [wire_distances1[i] + wire_distances2[i] for i in range(len(wire_distances1))]

In [10]:
closest_wire_distance = sorted(combined_wire_distances)[0]
print('The closest wire distance intersection is', closest_wire_distance)

The closest wire distance intersection is 25676
